In [1]:
from io import BytesIO
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

import csv
import datetime
import numpy as np
import pandas as pd
import requests
import time

#參考: 助教的Github

#STEP 1: 呼叫Chrome的Web driver
main_driver = webdriver.Chrome('/Users/pptaira/Documents/Fintech_2019S/chromedriver') 
main_driver.implicitly_wait(20)

#STEP 2: 打開目標網頁
main_driver.get('https://fred.stlouisfed.org/series/TEDRATE')
download_btn_ele = main_driver.find_element_by_xpath('//*[@id="download-button"]')
download_btn_ele.click()
excel_download_btn_ele = main_driver.find_element_by_xpath('//*[@id="fg-download-menu"]/li[1]/a')
head = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'}

#Wait一下，不然常常出HTTP Error或Attribute Error
main_driver.implicitly_wait(50)

url = excel_download_btn_ele.get_attribute('href')
df = pd.read_excel(url)
#確認目前結果
df.head(20)

,FRED Graph Observations,Unnamed: 1
0,Federal Reserve Economic Data,NaN
1,Link: https://fred.stlouisfed.org,NaN
2,Help: https://fred.stlouisfed.org/help-faq,NaN
3,Economic Research Division,NaN
4,Federal Reserve Bank of St. Louis,NaN
5,NaN,NaN
6,TEDRATE,"TED Spread, Percent, Daily, Not Seasonally Adj..."
7,NaN,NaN
8,Frequency: Daily,NaN
9,observation_date,TEDRATE


In [2]:
#STEP 3: 去掉9行以前的資料，重新排Index，改時間格式，改橫標
df = df.drop(df.index[[0 , 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9]])
df = df.reset_index(drop = True)
df['FRED Graph Observations'] = pd.to_datetime(df['FRED Graph Observations'] , format = "%Y%m%d")
df = df.rename(columns = {'FRED Graph Observations':'Date'})
df = df.rename(columns = {'Unnamed: 1':'Value'})

#確認結果
df.head(20)

,Date,Value
0,2014-03-10,0.18
1,2014-03-11,0.18
2,2014-03-12,0.18
3,2014-03-13,0.18
4,2014-03-14,0.18
5,2014-03-17,0.17
6,2014-03-18,0.18
7,2014-03-19,0.17
8,2014-03-20,0.17
9,2014-03-21,0.17
